In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine, types
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Build connection string safely
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

if not DB_PASSWORD:
    raise ValueError("DB_PASSWORD not set. Create a .env file.")

engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
)

file_path = "../data/raw/Sample - Superstore.csv"


try:
    df = pd.read_csv(file_path, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding="latin1")

print("Rows in dataframe:", len(df))

dtype_map = {
    "Row ID": types.Integer(),
    "Order ID": types.String(50),
    "Ship Mode": types.String(50),
    "Customer ID": types.String(20),
    "Customer Name": types.String(100),
    "Segment": types.String(50),
    "Country": types.String(50),
    "City": types.String(50),
    "State": types.String(50),
    "Postal Code": types.String(20),
    "Region": types.String(20),
    "Product ID": types.String(50),
    "Category": types.String(50),
    "Sub-Category": types.String(50),
    "Product Name": types.String(150),
}

df.to_sql(
    name="stg_superstore",
    con=engine,
    if_exists="replace",
    index=False,
    dtype=dtype_map
)

print("Staging table loaded with correct types")
